### GPU, cuda, gccの確認


In [ ]:
!nvidia-smi
!nvcc --version
!gcc --version

### パッケージのインストール / クリーンインストール
ssh先のjupyter環境などで実行の場合は, 当該環境にパッケージをインストール

In [ ]:
# クリーンが必要であれば実行
!rm -rf kineticEQ
#!rm -rf build

In [ ]:
# インストール
!git clone https://github.com/Minamium/kineticEQ.git
!pip install -e ./kineticEQ

### 動作確認

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import Image
import math

from kineticEQ import BGK2D2V_core

# =========================
# 1. シミュレーション設定
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"

nx, ny   = 256, 256
nv_x, nv_y = 16, 16

Lx, Ly   = 1.0, 1.0
vx_max, vy_max = 5.0, 5.0

dt       = 5e-5
T_total  = 5.0 
tau_tilde = 5e-4

# 初期モーメント場
x = torch.linspace(0.0, Lx, nx)
y = torch.linspace(0.0, Ly, ny)
X, Y = torch.meshgrid(x, y, indexing="ij")

n0 = torch.ones((nx, ny))
T0 = torch.ones((nx, ny))

# --- Kelvin–Helmholtz 型初期条件 ---

U0    = 0.5          # せん断層の代表速度
delta = 0.1 * Ly    # せん断層の厚み

# x 方向流速：y = Ly/2 まわりのタンhプロファイル
u_x0 = U0 * torch.tanh((Y - 0.5 * Ly) / delta)

# 小さい横方向擾乱（KH 成長用）
eps   = 0.1 * U0    # 擾乱の大きさ
k_x   = 2.0 * math.pi / Lx  # 1 波長
u_y0  = eps * torch.sin(k_x * X)


# =========================
# 2. インスタンス生成 & 実行
# =========================
bgk = BGK2D2V_core(
    nx=nx,
    ny=ny,
    nv_x=nv_x,
    nv_y=nv_y,
    dt=dt,
    T_total=T_total,
    Lx=Lx,
    Ly=Ly,
    vx_max=vx_max,
    vy_max=vy_max,
    tau_tilde=tau_tilde,
    n=n0,
    u_x=u_x0,
    u_y=u_y0,
    T=T0,
    device=device,
    dtype="float64",
    use_tqdm=True,
    flag_record_state=True,   # アニメーション用に時系列を記録
)

bgk.run_simulation()

states = bgk.animation_data
print(f"Recorded frames: {len(states)}")

# =========================
# 3. アニメーション (GIF) 作成
# =========================
fig, axes = plt.subplots(2, 2, figsize=(8, 6))
ax_n, ax_ux, ax_uy, ax_T = axes[0, 0], axes[0, 1], axes[1, 0], axes[1, 1]

Lx, Ly = bgk.Lx, bgk.Ly

# 0フレーム目で初期化
frame0 = states[0]
n_img  = ax_n.imshow(frame0["n"],  origin="lower", extent=[0, Lx, 0, Ly])
ux_img = ax_ux.imshow(frame0["u_x"], origin="lower", extent=[0, Lx, 0, Ly])
uy_img = ax_uy.imshow(frame0["u_y"], origin="lower", extent=[0, Lx, 0, Ly])
T_img  = ax_T.imshow(frame0["T"],  origin="lower", extent=[0, Lx, 0, Ly])

ax_n.set_title("Density n")
ax_ux.set_title("u_x")
ax_uy.set_title("u_y")
ax_T.set_title("Temperature T")

for ax in [ax_n, ax_ux, ax_uy, ax_T]:
    ax.set_xlabel("x")
    ax.set_ylabel("y")

fig.tight_layout()

def update(frame_idx):
    state = states[frame_idx]
    n_img.set_data(state["n"])
    ux_img.set_data(state["u_x"])
    uy_img.set_data(state["u_y"])
    T_img.set_data(state["T"])
    fig.suptitle(f"t = {state['time']:.3f}")
    return [n_img, ux_img, uy_img, T_img]

anim = animation.FuncAnimation(
    fig,
    update,
    frames=len(states),
    interval=100,   # 1フレーム 100 ms
    blit=False,
)

gif_name = "bgk2d2v_moments.gif"
anim.save(gif_name, writer=animation.PillowWriter(fps=10))
plt.close(fig)  # 静止画表示を抑制

# =========================
# 4. ノートブック上で GIF 再生
# =========================
Image(filename=gif_name)
